# TMD Model - New Parameters

From Javey 2019 Science Paper, but my own fitting

Initial stuff

In [1]:
%config InlineBackend.figure_format = 'retina' #to make inline images higher res

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from scipy.optimize import fsolve

np.seterr(divide = 'ignore') ;

Add my plot style in

In [2]:
plt.style.use(['presplot_small'])

Original TMD parameters

In [53]:
labels=['txr','txnr','ttr_n','ttnr_n','ttr_p','ttnr_p','Cbx','me','mh','Et','Eg','Vth']
tmds=['MoS_2','WS_2','MoSe_2','WSe_2']
params=np.zeros((4,np.size(labels)))

#original parameters
mos2=[8,np.inf,0.11,0.05,np.inf,np.inf,2e-11,0.0,3.5]
ws2=[2,np.inf,0.032,0.1,np.inf,np.inf,5e-12,0.0,2.4]
mose2=[80,26.7,4.4,0.066,50,50,3e-12,1e-10,0.03]
wse2=[16.7,1.9,0.33,0.033,1,1,2.5e-11,3.5e-10,0.06]

#new parameters
mos2=[8,np.inf,0.11,0.05,np.inf,np.inf,3.5,0.51,0.54,0.03,1.8,-15]
ws2=[2,np.inf,0.032,0.1,np.inf,np.inf,2.4,0.35,0.34,0.03,2.1,-15]
mose2=[80,26.7,4.4,0.066,50,50,0.03,0.50,0.58,0.03,1.5,-4]
wse2=[16.7,1.9,0.33,0.033,1,1,0.06,0.35,0.35,0.03,1.7,0]

params[0,:]=mos2
params[1,:]=ws2
params[2,:]=mose2
params[3,:]=wse2
all_plqy={}

Constants

In [54]:
k=1.381e-23 #J/K
t=300 #K
h=6.626e-34 #J/s
hbar=h/(2*np.pi)
m0=9.109e-31 #kg
q=1.602e-19 #coulombs
c=2.998e8 #m/s
eps0=8.85e-12 #F/m

Pick a TMD

In [85]:
tmd=3
V=-20
plqy=4e-4 # MoS2 at 20 V
plqy=4e-3 #WS2 at 20V
plqy=2e-4 #MoSe2 at 20V
plqy=3e-3 #MoSe2 at -20V
plqy=5e-4 #WSe2 at 20V
plqy=1e-2 #WSe2 at -20V

Calculate current lifetimes

In [86]:
#easier than making a dictionary - don't want to type in d every time
txr=params[tmd,0]*1e-9 #s
txnr=params[tmd,1]*1e-9 #s
ttr_n=params[tmd,2]*1e-6 #s - for some reason was reported in µs
ttnr_n=params[tmd,3]*1e-9 #s
ttr_p=params[tmd,4]*1e-6 #s - same as above
ttnr_p=params[tmd,5]*1e-9 #s
Cbx=params[tmd,6]
me=params[tmd,7]*m0
mh=params[tmd,8]*m0
Et=params[tmd,9]
Eg=params[tmd,10]
Vth=params[tmd,11]

#calculate total lifetimes for each quasiparticle
tx=1/(1/txnr+1/txr)
tt_n=1/(1/ttnr_n+1/ttr_n)
tt_p=1/(1/ttnr_p+1/ttr_p)

#calculate ni
gc2d=me/(np.pi*hbar**2)*1/(100**2) #1/cm^2*energy
gv2d=mh/(np.pi*hbar**2)*1/(100**2) #1/cm^2*energy
ni=(gc2d*gv2d)**0.5*(k*t)*np.exp(-q*Eg/(2*k*t))

#calculate T_n and T_p
mt_n=2*me+mh
mt_p=me+2*mh
mx=me+mh
T_n=np.pi*hbar**2/(k*t)*mt_n/(me*mx)*np.exp(q*Et/(k*t))*1e4
T_p=np.pi*hbar**2/(k*t)*mt_p/(me*mx)*np.exp(q*Et/(k*t))*1e4

#for initial N, P guesses
Cox=1/(1/(3.8*eps0*1e-2/50e-7)+1/(2*eps0*1e-2/100e-7)) #50 nm oxide=50e-7 cm; 8.85e-12F/m=8.85e-14F/cm
N0P0=np.abs(Cox/q*(V-Vth))

#solve for N,P
if V>Vth: #then we are electron doping, ef towards CB
    NPguess=[N0P0,ni**2/N0P0]

elif V<Vth: #then we are hole doping, ef towards VB
    NPguess=[ni**2/N0P0,N0P0]

elif V==Vth:
    NPguess=[ni,ni]


def NPfunc(y):
    #x[0]=N, x[1]=P

    #net charged carriers
    out = [Cox*(V-Vth)/q-(y[0]-y[1])]

    #2D mass action law. Modified becuase of v small & v big numbers!

    if NPguess[0]>0.01*gc2d*k*t: #N is large, P is very small – Taylor expand for P portion
        out.append(gc2d*k*t*(np.exp(y[0]/(gc2d*k*t))-1)*y[1]-ni**2)

    elif NPguess[1]>0.01*gv2d*k*t: #P is large, N is very small – Taylor expand for N portion
        out.append(y[0]*gv2d*k*t*(np.exp(y[1]/(gv2d*k*t))-1)-ni**2)

    else: #not degenerately doped
        out.append(y[0]*y[1]-ni**2)      

    return out

y0, infodict, ier, mesg = fsolve(NPfunc, NPguess, full_output=True)           

N=y0[0]
P=y0[1]

Calculate new nonradiative trion lifetime – assuming no radiative lifetime

In [83]:
ttnr_n_new=plqy*N*T_n/(1/txr-plqy/tx)
ttnr_n_new #in s

1.85791276365126e-11

In [84]:
1/txr/(1/tx+T_n*N/ttnr_n_new)

0.0005

In [87]:
ttnr_p_new=plqy*P*T_p/(1/txr-plqy/tx)
ttnr_p_new #in s

4.09889808895617e-10

In [88]:
1/txr/(1/tx+T_p*P/ttnr_p_new)

0.01

Calculate new radiative trion lifetime – assuming we know total lifetime, which is nonradiative dominated

In [70]:
ttr_n_new=T_n*N/(plqy*(1/tx+T_n*N/tt_n)-1/txr)
ttr_n_new

-2.593952501311112e-07

In [71]:
#check PLQY
(1/txr+T_n*N/ttr_n_new)/(1/tx+T_n*N/tt_n)

0.0002

In [72]:
tt_n

6.599901001484978e-11

In [73]:
ttnr_n

6.6e-11

In [74]:
ttr_n

4.4e-06

In [75]:
(1/txr+T_n*N/ttr_n)/(1/tx+T_n*N/tt_n)

0.00046894512081110956

In [76]:
(1/txr)/(1/tx+T_n*N/tt_n)

0.00045397255824896666

MoS2 - this gives 0.002 where we should get PLQY 0.0004. This shows that tt_n must be too long. Low enough PLQY cannot be achieved with any value of ttr_n. Should use previous method and fit for ttnr.

WS2 - this gives 0.01 where we should get PLQY 0.004. This shows that tt_n must be too long. Should use previous method and fit for ttnr.

In [ ]:
#new parameters
mos2=[8,np.inf,np.inf,0.008,np.inf,np.inf,3.5,0.51,0.54,0.03,1.8,-15]
ws2=[2,np.inf,np.inf,0.031,np.inf,np.inf,2.4,0.35,0.34,0.03,2.1,-20]
mose2=[80,26.7,np.inf,0.029,np.inf,0.3,0.03,0.50,0.58,0.03,1.5,-4]
wse2=[16.7,1.9,np.inf,0.018,np.inf,0.4,0.06,0.35,0.35,0.03,1.7,0]